# Notebook `05_app.ipynb`

---

# 🖥️ Déploiement de l'application interactive

Ce notebook final permet de **mettre en application les modèles de prédiction développés** dans les notebooks précédents.  
Il propose une **interface interactive Gradio** qui permet d'estimer :
- le **prix probable** d’un service Fiverr,
- ainsi que sa **tranche de prix** (Basse, Moyenne ou Haute).

Cette application s’appuie sur un **pipeline hybride** combinant embeddings de texte et variables numériques pour fournir des prédictions accessibles et rapides.

## 🎯 Objectifs

- 🧠 Intégrer les **fonctions de prédiction** basées sur les modèles enregistrés (`predict_price`, `predict_tranche`)
- 🧰 Concevoir une **interface utilisateur intuitive** avec `gradio.Blocks`
- 📝 Ajouter une fonction de **log** permettant d’enregistrer les prédictions pour analyse
- ⚙️ Proposer une interface flexible :
  - Sélection de la fiabilité par niveau prédéfini (`Acceptable`, `Bonne`, etc.)
  - Ou réglage manuel via un `slider` numérique
- 🚀 Déployer l'application en local ou sur un serveur

## 🧠 Fonctionnalités clés

- ✏️ **Entrée utilisateur** :
  - Titre du service (texte libre)
  - Niveau du vendeur (option désactivée ici)
  - Fiabilité (par choix prédéfini ou valeur personnalisée)

- 🔄 **Prétraitement transparent** :
  - Embedding de la description via `sentence-transformers`
  - Normalisation de la fiabilité
  - Alignement des colonnes avec les modèles entraînés

- 📈 **Sorties affichées** :
  - Prédiction du **prix** (`predict_price`)
  - Prédiction de la **tranche de prix** (`predict_tranche`)

- 📁 **Fonction de signalement** :
  - Ajout des prédictions dans un fichier `log.csv` pour audit ou analyse ultérieure

## ✅ Compétences mobilisées

- **Bloc 3 — C2** : Appliquer un pipeline de transformation cohérent entre entraînement et usage réel.
- **Bloc 5 — C4** : Déployer une application d’IA accessible et interactive (Gradio).

👨‍🏫 *Ce notebook constitue la version déployable de l’outil de prédiction, à présenter au jury ou à tester par les utilisateurs finaux.*

---

## 🧭 Sommaire

1. [📘 Importation des bibliothèques](#📘-1-importation-des-bibliothèques)
2. [📂 Préparation du fichier de log](#📂-2-préparation-du-fichier-de-log)
3. [🎚️ Définition des niveaux de fiabilité prédéfinis](#🎚️-3-définition-des-niveaux-de-fiabilité-prédéfinis)
4. [🔮 Fonction de prédiction principale](#🔮-4-fonction-de-prédiction-principale)
5. [📝 Fonction d’enregistrement des prédictions dans le log](#📝-5-fonction-denregistrement-des-prédictions-dans-le-log)
6. [🖥️ Interface utilisateur Gradio](#🖥️-6-interface-utilisateur-gradio)
7. [🔍 Analyse de sensibilité à la fiabilité](#🔍-7-analyse-de-sensibilité-à-la-fiabilité)
8. [📊 Analyse exploratoire : corrélation Fiabilité / Prix](#📊-8-analyse-exploratoire--corrélation-fiabilité--prix)

---

## 📘 1. Importation des bibliothèques

### 🧰 1.1. Pourquoi cette étape ?

Avant de lancer l’interface applicative, il est essentiel de charger les **bibliothèques nécessaires** au fonctionnement de Gradio, à la gestion des fichiers et à l’exécution des fonctions de prédiction.

Ces fonctions (`predict_price`, `predict_tranche`) ont été développées dans le fichier `predict.py` et reposent sur les modèles et outils enregistrés lors de la phase de modélisation.

### 📚 1.2. Bibliothèques utilisées

| Bibliothèque | Utilisation principale |
|--------------|------------------------|
| `gradio`     | Création de l’interface web interactive |
| `pandas`     | Gestion du fichier log (`log.csv`) |
| `os`         | Manipulation du système de fichiers |
| `predict`    | Import des fonctions de prédiction utilisées dans l’interface |

---

### 🐍 1.3. Script d’importation

In [17]:
import gradio as gr
import pandas as pd
import os
from predict import predict_price, predict_tranche  # doit être dans le même dossier ou accessible dans PYTHONPATH

---

## 📂 2. Préparation du fichier de log

### 🧾 2.1. Pourquoi cette étape ?

L’application permet aux utilisateurs de **signaler les prédictions** effectuées (prix et tranche) dans un fichier CSV de log.  
Ce fichier pourra être exploité ultérieurement pour :
- analyser les cas utilisateurs,
- affiner les modèles,
- ou justifier certaines prédictions en situation réelle.

Il est donc nécessaire de :
- définir un **chemin d’enregistrement** (`flagged/log.csv`),
- s’assurer que le répertoire `flagged/` existe pour éviter toute erreur d’écriture.

### 📁 2.2. Résultat attendu

- Le chemin `flagged/log.csv` est prêt à recevoir les enregistrements.
- Le dossier `flagged/` est créé s’il n’existait pas déjà.

---

### 🐍 2.3. Script de préparation du fichier log

In [18]:
LOG_PATH = "flagged/log.csv"
#os.makedirs(os.path.dirname(LOG_PATH), exist_ok=True)

---

## 🎚️ 3. Définition des niveaux de fiabilité prédéfinis

### 🧮 3.1. Pourquoi proposer des niveaux ?

L'utilisateur peut indiquer la **fiabilité du vendeur** soit :
- en **valeur libre** via un curseur (`Slider`),
- soit en sélectionnant un **niveau prédéfini**, plus intuitif et guidé.

Ces niveaux permettent :
- une **expérience utilisateur simplifiée**,
- une **standardisation** des entrées dans le cadre de tests ou d’évaluations.

### 📊 3.2. Correspondance des niveaux

| Niveau choisi    | Valeur numérique (%) |
|------------------|----------------------|
| Acceptable       | 80                   |
| Moyenne          | 85                   |
| Bonne            | 90                   |
| Très Bonne       | 96                   |
| Excellente       | 99                   |

---

### 🐍 3.3. Script de définition des niveaux de fiabilité

In [19]:
# 🔸 Niveaux prédéfinis de fiabilité (%)
choices = {
    "Acceptable": 80,
    "Moyenne": 85,
    "Bonne": 90,
    "Très Bonne": 96,
    "Excellente": 99
}

---

## 🔮 4. Fonction de prédiction principale

### 🧠 4.1. Objectif

Cette fonction est le **cœur du système de prédiction**.  
Elle combine :
- le choix de la **source de fiabilité** (niveau prédéfini ou valeur libre),
- la **prédiction du prix exact** (modèle de régression),
- la **prédiction de la tranche de prix** (modèle de classification).

Elle assure également la **gestion des erreurs** en cas de mauvaise saisie ou de problème technique.

### ⚙️ 4.2. Paramètres

| Paramètre           | Type     | Description                                                                 |
|---------------------|----------|-----------------------------------------------------------------------------|
| `description`        | `str`    | Texte décrivant le service Fiverr                                          |
| `niveau`             | `str`    | Niveau du vendeur (actuellement non utilisé dans le modèle)                |
| `use_predefined`     | `bool`   | Indique si on utilise un niveau de fiabilité prédéfini                     |
| `fiabilite_percent`  | `float`  | Valeur manuelle de fiabilité (%) si `use_predefined` est `False`          |
| `fiabilite_choix`    | `str`    | Nom du niveau de fiabilité choisi si `use_predefined` est `True`           |

### 🎯 4.3. Résultat

Retourne deux prédictions :
- le **prix estimé** (float, arrondi à 2 décimales),
- la **tranche de prix** (`Basse`, `Moyenne`, `Haute`).

En cas d'erreur, un message d'alerte est retourné à la place du prix.

---

### 🐍 4.4. Script de prédiction principale

In [20]:
# 🔍 Fonction principale de prédiction
def faire_une_prediction(description, niveau, use_predefined, fiabilite_percent, fiabilite_choix):
    fiabilite = (choices[fiabilite_choix] if use_predefined else fiabilite_percent) / 100
    try:
        prix = predict_price(description, fiabilite)
        tranche = predict_tranche(description, fiabilite)
        return round(prix, 2), tranche
    except Exception as e:
        return f"Erreur : {str(e)}", ""

---

## 📝 5. Fonction d’enregistrement des prédictions dans le log

### 🧠 5.1. Objectif

Cette fonction permet de **conserver un historique des prédictions** réalisées via l’application Gradio.  
Elle enregistre chaque estimation dans un fichier `log.csv`, avec les données d’entrée et les résultats associés.

### 🗃️ 5.2. Données enregistrées

| Colonne            | Description                                             |
|--------------------|---------------------------------------------------------|
| `Description`       | Texte fourni par l’utilisateur                          |
| `Niveau`            | Niveau du vendeur sélectionné (actuellement non utilisé) |
| `Fiabilité (%)`     | Valeur de fiabilité (selon sélection manuelle ou prédéfinie) |
| `Prix prédit (€)`   | Résultat de la prédiction de prix (régression)         |
| `Tranche prédite`   | Résultat de la classification (`Basse`, `Moyenne`, `Haute`) |

### 🧾 5.3. Comportement

- Le fichier `log.csv` est **créé s’il n’existe pas**, ou mis à jour en **mode ajout** (`append`).
- Chaque prédiction est **enregistrée sur une ligne distincte**.

> ℹ️ Ce fichier log peut servir de **trace d’utilisation**, ou de base pour des analyses a posteriori.

---

### 🐍 5.4. Script d’enregistrement dans le fichier log

In [21]:
# 📝 Fonction d'enregistrement dans le fichier log
def enregistrer_log(description, niveau, use_predefined, fiabilite_percent, fiabilite_choix, prix, tranche):
    fiabilite = choices[fiabilite_choix] if use_predefined else fiabilite_percent
    log_data = {
        "Description": description,
        "Niveau": niveau,
        "Fiabilité (%)": fiabilite,
        "Prix prédit (€)": prix,
        "Tranche prédite": tranche
    }

    df_log = pd.DataFrame([log_data])
    if os.path.exists(LOG_PATH):
        df_log.to_csv(LOG_PATH, mode="a", index=False, header=False)
    else:
        df_log.to_csv(LOG_PATH, index=False)

    return "✅ Signalement enregistré avec succès."

---

## 🖥️ 6. Interface utilisateur Gradio

### 🎯 6.1. Objectif

Proposer une **interface interactive et intuitive** pour permettre à l’utilisateur :
- de saisir les informations d’un service Fiverr,
- d’obtenir immédiatement une **estimation du prix** (`Régression`),
- et sa **tranche de prix** (`Classification`),
- puis de **signaler** le cas au fichier `log.csv` si besoin.

### 🧰 6.2. Composants principaux

| Élément                    | Fonction                                                                 |
|----------------------------|--------------------------------------------------------------------------|
| `Textbox` (description)    | Entrée textuelle décrivant le service                                   |
| `Dropdown` (niveau)        | Choix du niveau du vendeur (actuellement masqué, future évolution)      |
| `Checkbox` / `Radio`       | Sélection du niveau de fiabilité (manuelle ou prédéfinie)               |
| `Button`                   | Lance la prédiction (`📈 Estimer le prix`)                              |
| `Textbox` (sorties)        | Affichage des résultats (`Prix estimé`, `Tranche estimée`)              |
| `Button`                   | Enregistrement de la prédiction dans le fichier log (`🚨`)              |
| `Textbox` (confirmation)   | Message de confirmation en cas de signalement réussi                    |

### 🔁 6.3. Logique d’interaction

- **Fiabilité personnalisée ou prédéfinie** :  
  L’utilisateur peut choisir d’utiliser un curseur libre (pourcentage) ou un niveau de fiabilité prédéfini.

- **Synchronisation dynamique** :  
  Le radio bouton met à jour la valeur du slider et inversement si on décoche la case.

- **Traitement centralisé** :  
  Les fonctions `faire_une_prediction` et `enregistrer_log` pilotent la logique métier.

---

### 🐍 6.4. Script Gradio (création de l’interface)

In [22]:

# 🖥️ Interface utilisateur Gradio
with gr.Blocks() as iface:
    gr.Markdown("""
        ## 🎯 Application de prédiction de prix Fiverr
        Cette application permet d’estimer automatiquement :
        - 💰 Le **prix probable** d’un service publié sur Fiverr,
        - 📊 Sa **tranche de prix** parmi trois catégories (Basse / Moyenne / Haute).
        Elle s’appuie sur un pipeline hybride combinant des embeddings de description et des variables numériques.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            description = gr.Textbox(
                label="✏️ Titre du service",
                placeholder="Ex : Je fais le ménage",
                value="Je fais le ménage"  # Valeur par défaut
            )
            niveau =  gr.Dropdown(label="🔰 Niveau du vendeur", choices=["Beginner", "Intermediate", "Expert"], value="Beginner", visible=False)
            use_predefined = gr.Checkbox(label="🎛️ Utiliser les niveaux prédéfinis de fiabilité", value=True)

            fiabilite_percent = gr.Slider(label="Fiabilité (%)", minimum=0, maximum=100, value=80, step=5, visible=False)
            fiabilite_choix = gr.Radio(
                label="🎚️ Choisissez un niveau de fiabilité",
                choices=list(choices.keys()),
                value="Acceptable",
                visible=True
            )

            def sync_slider_with_radio(choix):
                return gr.update(value=choices[choix])

            fiabilite_choix.change(sync_slider_with_radio, inputs=fiabilite_choix, outputs=fiabilite_percent)

            def toggle_inputs(use_predef):
                return {
                    fiabilite_percent: gr.update(visible=not use_predef),
                    fiabilite_choix: gr.update(visible=use_predef)
                }

            use_predefined.change(toggle_inputs, inputs=use_predefined, outputs=[fiabilite_percent, fiabilite_choix])

            bouton_predire = gr.Button("📈 Estimer le prix")

        with gr.Column(scale=1):
            sortie_prix = gr.Textbox(label="💰 Prix estimé")
            sortie_tranche = gr.Textbox(label="📊 Tranche estimée")
            bouton_signaler = gr.Button("🚨 Ajouter au fichier log.csv")
            confirmation = gr.Textbox(label="✅ Confirmation", visible=False)

            bouton_predire.click(
                fn=faire_une_prediction,
                inputs=[description, niveau, use_predefined, fiabilite_percent, fiabilite_choix],
                outputs=[sortie_prix, sortie_tranche]
            )

            bouton_signaler.click(
                fn=enregistrer_log,
                inputs=[description, niveau, use_predefined, fiabilite_percent, fiabilite_choix, sortie_prix, sortie_tranche],
                outputs=confirmation
            )

# 🚀 Lancement
iface.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


IMPORTANT: You are using gradio version 3.39.0, however version 4.44.1 is available, please upgrade.
--------


---

## 🔍 7. Analyse de sensibilité à la fiabilité

### 🧪 7.1. Objectif de cette cellule

Vérifier que la **variable `Fiabilité` a bien un impact** significatif sur le prix prédit.  
On fait varier la valeur de fiabilité de 0 % à 100 %, tout en gardant constante la description du service.

### 📊 7.2. Méthodologie

- Le service testé est simplement : `"Je fais le ménage"`
- On fait évoluer la fiabilité de 0.00 à 1.00 (i.e. de 0 % à 100 %)
- Pour chaque niveau de fiabilité, on **prévoit le prix** avec le modèle de régression finalisé
- On affiche ensuite les résultats dans la console

### 🔍 7.3. Ce que l'on cherche à observer

- Une **augmentation globale du prix** avec la fiabilité croissante (logique métier)
- Une **zone d'inflexion ou palier éventuel** aux alentours de 0.8 (80 %) — critère de haute fiabilité
- Des **résultats cohérents** avec ceux observés dans l’application Gradio

> 💡 *Cette étape permet de valider qualitativement la sensibilité du modèle aux variations d’un indicateur clé métier.*

---

### 🐍 7.4. Script de test d’impact de la fiabilité

In [23]:
# Exemple de description et niveau à tester
desc = "Je fais le ménage"

# Test de l’impact de la fiabilité sur la prédiction du prix
for f in [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.81, 0.82, 0.83, 0.84, 0.85, 0.9, 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99, 1.0]:
    prix = predict_price(desc, f)
    print(f"Fiabilité = {f*100:.0f}% → Prix prédit = {prix} €")

Fiabilité = 0% → Prix prédit = 13.8 €
Fiabilité = 10% → Prix prédit = 13.8 €
Fiabilité = 20% → Prix prédit = 13.8 €
Fiabilité = 30% → Prix prédit = 13.8 €
Fiabilité = 40% → Prix prédit = 13.8 €
Fiabilité = 50% → Prix prédit = 13.8 €
Fiabilité = 60% → Prix prédit = 13.8 €
Fiabilité = 70% → Prix prédit = 13.8 €
Fiabilité = 80% → Prix prédit = 13.8 €
Fiabilité = 81% → Prix prédit = 13.8 €
Fiabilité = 82% → Prix prédit = 13.8 €
Fiabilité = 83% → Prix prédit = 13.8 €
Fiabilité = 84% → Prix prédit = 14.2 €
Fiabilité = 85% → Prix prédit = 14.2 €
Fiabilité = 90% → Prix prédit = 14.89 €
Fiabilité = 91% → Prix prédit = 14.89 €
Fiabilité = 92% → Prix prédit = 14.89 €
Fiabilité = 93% → Prix prédit = 14.89 €
Fiabilité = 94% → Prix prédit = 15.36 €
Fiabilité = 95% → Prix prédit = 15.89 €
Fiabilité = 96% → Prix prédit = 16.15 €
Fiabilité = 97% → Prix prédit = 16.76 €
Fiabilité = 98% → Prix prédit = 17.32 €
Fiabilité = 99% → Prix prédit = 17.32 €
Fiabilité = 100% → Prix prédit = 17.2 €


---

## 📊 8. Analyse exploratoire : corrélation Fiabilité / Prix

### 🧪 8.1. Objectif de cette cellule

Vérifier empiriquement si la **fiabilité du vendeur est corrélée au prix proposé**.  
Cela permet de **valider la pertinence** de cette variable comme feature dans notre modèle.

### 🧮 8.2. Statistiques de la variable `Fiabilite`

- Affichage de la distribution statistique (`min`, `max`, `mean`, `std`, etc.)
- Vérification que la variable **varie suffisamment** pour avoir un potentiel explicatif

### 🔗 8.3. Corrélation entre `Fiabilite` et `Prix`

- Calcul du **coefficient de corrélation de Pearson**
- Permet de détecter une **relation linéaire potentielle** entre la variable explicative et la cible

> 💡 Une corrélation significative (positive ou négative) soutient l'idée que cette variable a un **rôle déterminant** dans le modèle de prédiction.

---

### 🐍 8.4. Script d’analyse exploratoire

In [24]:
df = pd.read_csv("data/fiverr_cleaned.csv")
print(df["Fiabilite"].describe())
print(df[["Fiabilite", "Prix"]].corr())

count    1252.000000
mean        0.739307
std         0.125703
min         0.081877
25%         0.698787
50%         0.789447
75%         0.826738
max         0.873558
Name: Fiabilite, dtype: float64
           Fiabilite      Prix
Fiabilite   1.000000 -0.018182
Prix       -0.018182  1.000000
